# API `Functional` de tf.keras

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thalitadru/CoursNNDL/blob/master/TFKerasFunctional.ipynb)

tf.keras est l'API haut-niveau par défaut de Tensorflow. Pour la plupart des projets, elle sera souvent suffisante pour exprimer vos modèles, avec l'avantage d'éliminer pas mal de code répétitif "boiler-plate" avec pas mal d'abstractions sur les solveurs d'optimisation (`optimisers`) et sur boucles d'entraînement et validation (`model.fit()` et `model.evaluate()`).
Il s'agit en plus d'une API bien documentée et réputée pour sa facilité de prise en main.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, losses

SEED = 0
np.random.seed(SEED)
rng = np.random.RandomState(SEED)
tf.random.get_global_generator().reset_from_seed(SEED)

## Functional API

Avec l'API fonctionnele vous peuvez creer des modèles plus complexes. Vous allez enchainner les opèrations correspondantes à chaque couche, puis fournir les tenseurs d'entree et de sortie comme paramètres pour la construction d'un `tf.keras.Model`.

### Inputs

In [ ]:
img_inputs = keras.Input(shape=(32, 32, 3))

In [ ]:
img_inputs.dtype

In [ ]:
img_inputs.shape

#### Question

Pourquoi pensez-vous que le tenseur `img_shape` a une dimension de taille `None` ?

### Calculs intermediaires
Nous pouvons connecter des couches avec plus de liberté.

In [ ]:
conv1 = layers.Conv2D(
    filters=16,
    kernel_size=3,
    activation="relu",
    input_shape=(32, 32, 3),
    padding="same",
    strides=2,
    kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
    name="conv1"
)(img_inputs)
conv1

In [ ]:
conv2a = layers.Conv2D(
    filters=16,
    kernel_size=3,
    activation="relu",
    padding="same",
    kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
    name='conv2a'
)(conv1)
conv2a

In [ ]:
conv2b = layers.Conv2D(
    filters=16,
    kernel_size=5,
    activation="relu",
    padding="same",
    kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
    name='conv2b'
)(conv1)
conv2b

In [ ]:
conv2 = layers.Concatenate(axis=-1)([conv2a, conv2b])
conv2

In [ ]:
conv3 = layers.Conv2D(
    filters=16,
    kernel_size=3,
    activation="relu",
    padding="valid",
    strides=2,
    kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
    name="conv3"
)(conv2)
conv3

In [ ]:
x = layers.Flatten()(conv3)
x

In [ ]:
dense1 = layers.Dense(
    units=64,
    kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
    name="dense1",
)(x)
dense1

In [ ]:
dense2 = layers.Dense(
    32, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED), name="dense2"
)(dense1)
dense2

### Output
La dernière couche est déclaré comme les autres. Il faut penser a y mettre une taille adapté au problème. Si c'est de la classification 100 classes par exemple:

In [ ]:
outputs = layers.Dense(100)(dense2)
outputs

### Modèle

In [ ]:
model2 = keras.Model(inputs=img_inputs, outputs=outputs, name="my_cnn")
model2

In [ ]:
model2.summary()

#### Visualisation

In [ ]:
keras.utils.plot_model(model2)

In [ ]:
try:
    import visualkeras as vk
except ModuleNotFound:
    !pip install visualkeras
vk.layered_view(model2, legend=True)

## Exercice: Classification d'images sur Cifar100

Entraînez ce modèle sur le dataset Cifar100
- Chargez les données
- Compilez le modèle
- Appelez `fit` pour lancer l'entrainement
- Affichez les courbes d'apprentissage

In [ ]:
#?keras.datasets.cifar10.load_data

In [ ]:
cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
# compilez le modèle avec les parametres
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = 'RMSprop'
metrics = ['accuracy']

In [ ]:
model2.compile(optimizer, loss, metrics)
out = model2.fit(
    train_images,
    y=train_labels,
    epochs=2,
    batch_size=16,
    validation_split=0.2,
)

In [ ]:
model2.evaluate(test_images, test_labels)

In [ ]:
def learning_curves(out, **kwargs):
    metrics = out.history.keys()
    metrics = list(filter(lambda m: not m.startswith("val"), metrics))
    num_metrics = len(metrics)
    fig, axs = plt.subplots(
        num_metrics,
        1,
        figsize=(15, 5 * num_metrics),
    )
    for i, metric in enumerate(metrics):
        loss = out.history[metric]
        val_loss = out.history["val_" + metric]
        epochs_range = range(1, len(loss) + 1)
        ax = axs[i]
        ax.plot(
            epochs_range,
            loss,
            marker=".",
            linestyle="dashed",
            label="Train " + metric,
            **kwargs
        )
        ax.plot(
            epochs_range,
            val_loss,
            marker=".",
            linestyle="dashed",
            label="Valid " + metric,
            **kwargs
        )
        ax.legend()
        ax.set_title(metric)
    ax.set_xlabel("epochs")
    fig.suptitle("courbes d'apprentisage x époques", fontsize="x-large")

In [ ]:
learning_curves(out)

## Avancé : API orienté-objet pour Modèles et couches custom
On peut créer un modèle personalisé en héritant de `tf.keras.Model` (voir [doc](https://www.tensorflow.org/guide/keras/custom_layers_and_models#the_model_class)). 

Il est également possible de creer des couches personalisées en héritant de `tf.keras.layers.Layer` (voir [doc](https://www.tensorflow.org/guide/keras/custom_layers_and_models#the_layer_class_the_combination_of_state_weights_and_some_computation)).

Les classes dérivés doivent implémenter une méthode `_call_(self, input)` qui compute les calculs en avant  (forward pass) du modèle, à l'aide de l'API fonctionnelle, et retourne les outputs